Importing the liberaries to work with

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics

Here we are performing a rediction on dataset which wil requir some modifications to the original data and we need to perform feature engineering on it. so let's start the journey!!

Importing the datasets

In [ ]:
dataset_train = pd.read_csv("../input/loan-prediction-practice-av-competition/train_csv.csv")
dataset_test = pd.read_csv("../input/loan-prediction-practice-av-competition/test.csv.csv")
submission = pd.read_csv("../input/loan-prediction-practice-av-competition/sample_submission.csv")

dataset_train.info()

Here is the information on dataset we have to train on!


In [ ]:
dataset_test.info()

Here is the information on dataset we have to test on!

The test dataset have column ['Loan_Status'] missing which we have to predict!!

In [ ]:
dataset_train.isnull().sum()

In [ ]:
################################################################################################################## Missing Values
dataset_train = dataset_train.dropna(subset = ['Gender', 'Married', 'LoanAmount', 'Loan_Amount_Term'])
dataset_train['Dependents'].fillna(dataset_train['Dependents'].mode()[0], inplace = True) 
dataset_train['Self_Employed'].fillna(dataset_train['Self_Employed'].mode()[0], inplace = True) 
dataset_train['Credit_History'].fillna(dataset_train['Credit_History'].mode()[0], inplace = True)
dataset_test = dataset_test.dropna(subset = ['Gender', 'Married', 'LoanAmount', 'Loan_Amount_Term'])
dataset_test['Dependents'].fillna(dataset_test['Dependents'].mode()[0], inplace = True) 
dataset_test['Self_Employed'].fillna(dataset_test['Self_Employed'].mode()[0], inplace = True) 
dataset_test['Credit_History'].fillna(dataset_test['Credit_History'].mode()[0], inplace = True)
dataset_train['Total_Income'] = dataset_train['ApplicantIncome'] + dataset_train['CoapplicantIncome']
dataset_test['Total_Income'] = dataset_test['ApplicantIncome'] + dataset_test['CoapplicantIncome']

In [ ]:
##################################################################################################################Finding and removing outliers
def outlier_remover(dataset, column):
    q1 = dataset[column].quantile(0.25)
    q3 = dataset[column].quantile(0.75)
    iqr = q3 - q1
    dataset.drop(dataset[(dataset[column] < (q1 - 1.5 * iqr)) | (dataset[column] > (q3 + 1.5 * iqr))].index, inplace = True)
    return

outlier_remover(dataset_train, 'Total_Income')
outlier_remover(dataset_test, 'Total_Income')

The above function removes outlier using the IQR score method

In [ ]:
dataset_train['Loan_Status'].value_counts().plot.bar()
plt.show()

Here We can see the ration of loan status aproved vs rejected!!

In [ ]:
################################################################################################################## Dealing with categories
def replacer_catagory(points, data, items):
    for i in range (0, points):
        dataset_train.replace(items[i], i, inplace=True)
        dataset_test.replace(items[i], i, inplace=True)
    return

The above function perform like category encoding and replacing values in same column with numbers

In [ ]:
################################################################################################################## Percentage caculator 
def percentage_calc(data1, data2):
    percentage = (data1 / data2) * 100
    return percentage

The above function calculates the percentages upon calling

In [ ]:
################################################################################################################## Difference graph plotter
def difference_plotter(points, data, plt_title, plt_diff, items):
    dataset_train[data].value_counts().plot.bar(title = plt_title)
    percent_approval = []
    relevance = 0
    replacer_catagory(points, data, items)
    for i in range (0, points):
        x = dataset_train[data][(dataset_train['Loan_Status'] == 'Y')].value_counts()[i]
        y = dataset_train[data].value_counts()[i]
        percent_approval.insert(i, percentage_calc(x, y))
        print("Percentage of loans approved of ", plt_diff[i], " :-> ", int(percent_approval[i]), '%')
    for i in range (0, points - 1):
        relevance = percent_approval[i] - percent_approval[i + 1]
        if relevance >= 0:
            temp = plt_diff[i]
        else:
            temp = plt_diff[i + 1]
            relevance = relevance * -1
        if relevance >= 10:
            conclusion = "Which is high and one should be ", temp, " In order to higher the chances of getting the loan approved"
        if (relevance < 10) & (relevance >= 5):
            conclusion = "Which is average and is not mandatory ", temp, " In order to higher the chances of getting the loan approved"
        if relevance <= 5:
            conclusion = "Which is low and completely optional that one should be ", temp
        print("The difference in loan Approvals is :-> ", int(relevance), "% ", conclusion)
    plt.show()
    return

The above function plots a differentiating graph and responds to the amount of differences between variables!!

In [ ]:
#########################################################Plot based on Genders
difference_plotter(2, 'Gender', 'Type of Gender', ['Male', 'Female'], ['Male', 'Female'])

In [ ]:
#########################################################Plot based on Education
difference_plotter(2, 'Education', 'Type of Education', ['Graduated', 'Unraduated'], ['Graduate', 'Not Graduate'])

In [ ]:
#########################################################Plot based on self employment
difference_plotter(2, 'Self_Employed', 'Self Employment', ['Self_Employed', 'Employed by others'], ['Yes', 'No'])

In [ ]:
#########################################################Plot for Marrital Status
difference_plotter(2, 'Married', 'Marrital Status', ['Married', 'Un-Married'], ['Yes', 'No'])

In [ ]:
#########################################################Plot for Marrital Status
difference_plotter(4, 'Dependents', 'Number of Dependents', ['1 Dependents', '2 Dependents', '3 Dependents', '3+ Dependents'], ['0', '1', '2', '3+'])

In [ ]:
#########################################################Plot for Credit History
difference_plotter(2, 'Credit_History', 'Credit History', ['Non Credit History', 'Credit History'], ['1', '0'])

In [ ]:
#########################################################Plot for Property Area
difference_plotter(3, 'Property_Area', 'Property Area', ['Rural', 'Semi-Urban', 'Urban'], ['Rural', 'Semiurban', 'Urban'])

In [ ]:
######################################################### Histogram maker
def histogram_making(data):
    dataset_train[data].hist(bins = 50)
    print('Median value of ' + data +  ' is  :', dataset_train[data].median())
    plt.show()
    return

Above function makes histograms for data provided

In [ ]:
#########################################################Plot for Applicants Income
histogram_making('ApplicantIncome')

In [ ]:
#########################################################Plot for Coapplicant Income
histogram_making('CoapplicantIncome')

In [ ]:
#########################################################Plot for Loan Amount
histogram_making('LoanAmount')

In [ ]:
######################################################### Group histogram maker
def group_histogram_making(bins, groups, data, x_label, conclusion):
    dataset_train['bins_data'] = pd.cut(dataset_train[data], bins, labels = groups)
    Income_bin = pd.crosstab(dataset_train['bins_data'], dataset_train['Loan_Status']) 
    Income_bin.div(Income_bin.sum(1).astype(float), axis = 0).plot(kind = "bar", stacked = True)
    plt.xlabel(x_label) 
    plt.ylabel('Percentage')
    plt.show()
    print(conclusion)
    return

Above function creates grouped histograms with bins data

In [ ]:
#########################################################Plot for Applicants Income vs Loan Status
group_histogram_making([0, 1000, 2000, 4000 ,6000, 81000], ['Low', 'Below Average','Average','High', 'Very high'], 'ApplicantIncome', 'Applicant Income', "The Income should be above 2000 in order to get loan approved with ease")

In [ ]:
#########################################################Plot for CoapplicantStatus Income vs Loan 
group_histogram_making([0, 1000, 2000, 3000, 42000], ['Low', 'Average', 'High', 'Very High'], 'CoapplicantIncome', 'Coapplicant Income', "The Compliant graph shows that lower income are better in getting loans approved, which is a bit not right.\nThis result can be due to some missing factor\nLets make a new factor to recognise this behaviour")

In [ ]:
#########################################################Plot for Total Income vs Loan Status
group_histogram_making([0, 2000, 4000 ,6000, 81000], ['Low','Average','High', 'Very high'], 'Total_Income', 'Total Income', "The Income should be above 2000 in order to get loan approved with ease\nThis Behaviour is acceptable and firmly holds the conclusion predicted.")

In [ ]:
#########################################################Plot for Loan Amount vs Loan Status
group_histogram_making([0, 100, 200, 700], ['Low', 'Average', 'High'], 'LoanAmount', 'Loan Amount', "The low amounts of loans get approved more than the higher amounts but the difference is small to be baising factor")
dataset_train = dataset_train.drop(['bins_data'], axis = 1)

In [ ]:
######################################################### Plot for Finding corellations between variables
dataset_train['Dependents'].replace('3+', 3,inplace=True)
dataset_train['Loan_Status'].replace('N', 0,inplace=True)
dataset_train['Loan_Status'].replace('Y', 1,inplace=True)
matrix = dataset_train.corr()
sns.heatmap(matrix, vmax = 1, square = True)
plt.show()
print("High Correlation :-> Applicant Income & Loan Amount\nHigh Correlation :-> Credit History & Loan Amount\n")
print("Average Correlation :-> Compplicant Income & Loan Amount\nAverage Correlation :-> Dependents & Loan Amount\n")

Now that all the data visualisation is erformed and our data analytics part is completed. the last thing befor predicting the status and amount is to approximate values which will help reduce randomness of data and provide some beter scores in matchine learner model.

In [ ]:
##################################################################################################################Scaling Total Income and Loan Amount for prediction
def approximator(dataset, column, approximation):
    rounded = []
    j = 0
    for i in dataset[column].items():
        j = approximation * round(i[1] / approximation)
        rounded.append(j)
    dataset[column] = rounded
    return

approximator(dataset_train, 'Total_Income', 500)
approximator(dataset_test, 'Total_Income', 500)

This function approximates the values bu certain amount provided

# Model for prediction

In [ ]:
################################################################################################################## Model Building for training
def get_dataset(dataset_y, dataset_x):
    y = dataset_y.values
    X = dataset_x
    
    cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)
    scores = cross_val_score(RandomForestClassifier(), X, y, scoring = 'accuracy', cv = cv, n_jobs = -1, error_score = 'raise')
    print('%.3f (%.3f)' % (mean(scores), std(scores)))
    
    model_test = RandomForestClassifier()
    model_test.fit(X, y)
    ypred = model_test.predict(dataset_test.drop(['Loan_ID'], 1))
    return ypred, model_test

This above function is used to train the model on dataset_train data

In [ ]:
################################################# get database and plot the result of all models & predicting on new dataset
ypred, model_test = get_dataset(dataset_train['Loan_Status'], pd.get_dummies(dataset_train.drop(['Loan_ID', 'Loan_Status'], 1)))
submission['Loan_ID'] = dataset_test['Loan_ID']
submission['Loan_Status'] = ypred
submission['Loan_Status'].replace({0 : 'N', 1 : 'Y'}, inplace = True)

This is the prediction percentage accuracy of Loan Status predictions on train dataset which is 79.9% with standard daviation of 3.3%.
The predicted loan Status are stored in submission dataset

In [ ]:
approximator(dataset_test, 'LoanAmount', 100)
approximator(dataset_train, 'LoanAmount', 100)

ypred, model_test = get_dataset(dataset_train['LoanAmount'], pd.get_dummies(dataset_train.drop(['Loan_ID', 'LoanAmount'], 1)))
confusion_matrix = pd.DataFrame(dataset_test['LoanAmount'])
confusion_matrix['predicted'] = ypred
sns.heatmap(metrics.confusion_matrix(confusion_matrix['LoanAmount'], confusion_matrix['predicted'], labels = confusion_matrix['LoanAmount'].unique()), vmax = 100, annot = True, fmt='g')

This is the prediction percentage accuracy of Loan Amount predictions on test dataset which is 71.8% with standard daviation of 4.8%. The predicted loan Amount and the original Loan Amount are shown with the hep of confustion matrix